# DDoS and Intrusion detection


In this exercise, we'll use Spark structured streaming to detect DDoS attacks and attempts to access the admin panel of the website.

* Use the [fake-ddos](fake-ddos.ipynb) notebook to simulate a DDoS attack.
* Use the [fake-intrusion](fake-intrusion.ipynb) notebook to simulate an intrusion attempt.

In [ ]:

# Ensure the required Python 3 dependencies are installed.
import sys
!{sys.executable} -m pip uninstall -y pyarrow # https://github.com/jupyter/docker-stacks/issues/921
!{sys.executable} -m pip install kafka-python pyarrow

Create a Spark context and specify that the python spark-kafka libraries need to be added.

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.3 pyspark-shell'

import pyspark 
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils


sc = SparkContext()
sc.setLogLevel("WARN")
spark = SparkSession(sc)

Create a streaming DataFrame that represents the events received from the Kafka topic `clicks-cleaned`.

Cast the json to columns in the DataFrame. Make sure to use TimestampType for the `ts_ingest` since we already converted it in the `clean` notebook.

Create a [user-defined function (`udf`)](https://docs.databricks.com/spark/latest/spark-sql/udf-python.html) `forbidden_clicks` which takes a URL as input and returns `True` if the URL points to the admin part of the website (when it ends with `/admin`).

As an example, the following code creates a UDF which squares each value of a column. It is used on the "id" column and the resulting column's name is changed to "id_squared".

```python
from pyspark.sql.functions import udf
@udf("long")
def squared_udf(s):
  return s * s
df = spark.table("test")
display(df.select("id", squared_udf("id").alias("id_squared")))
```

Use the UDF to create the dataframe `df_forbidden` which contains the collumn `forbidden` which specifies if the URL is an admin URL.

We'll do the same for detecting ddos attacks. First we want to flag whether an individual event is suspicious, i.e. whether the page_timer and page_height are both `0`. However, this time we'll use a `pandas_udf`.

[Regular Python UDF's have the disadvantage that they operate on one row at a time](https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html), causing them to suffer from high serialization and invocation overhead. Pandas UDF's are built on top of Apache Arrow to support high-performant UDF's in Python.

This is the squared_udf converted to a pandas udf.

```python
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf('long', PandasUDFType.SCALAR)
def squared_pandas_udf(s):
    return s * s

df = spark.table("test")
display(df.select("id", squared_udf("id").alias("id_squared")))
```

The regular UDF version works one row at a time: the user-defined function takes a long `s` and returns the result of `s*s` as a long. In the Pandas version, the user-defined function takes a pandas.Series `s` and returns the result of `s*s` as a pandas.Series. Because `s*s` is vectorized on `pandas.Series`, the Pandas version is much faster than the row-at-a-time version.

Note that there are two important requirements when using scalar pandas UDFs:

* The input and output series must have the same size.
* How a column is split into multiple pandas.Series is internal to Spark, and therefore the result of user-defined function must be independent of the splitting.


In the cell above we highlight the use of high performance UDF's with pandas. For simple use cases such as the one here we could also avoid using UDF's and write the following instead:

```python
df_ddos = df_data.withColumn('flagged', when((col('visitor_page_timer') == 0) & (col('visitor_page_height') == 0), True).otherwise(False))
```

The second step in detecting a ddos attack is counting how many suspicious events happen within a certain timeframe. For this, well combine `groupBy` and a 30 seconds `window` based on the `ts_ingest` timestamp.

Now run these queries and write the output to `clicks-calculated-forbidden` and `clicks-calculated-ddos`. Use a trigger with `processingTime = "30 seconds"` for the ddos query so that the next interval is only calculated 30 seconds after the first interval starts.

When multiple streaming queries are started in the same program we have to use `spark.streams.awaitAnyTermination()`. Otherwise only the first query will be executed.

In [ ]:
spark.streams.awaitAnyTermination()